In [1]:
from keras.datasets import boston_housing

C:\Users\coroi\Anaconda3\envs\data-science\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\coroi\Anaconda3\envs\data-science\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to

In [36]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data[0])

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

print(train_data[0])
test_data -= mean
test_data /= std



[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
  0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
  0.8252202 ]


In [43]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    
    model.add(layers.Dense(64, activation="relu", input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation="relu"))
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer="rmsprop",
        loss="mse",
        metrics=["mae"]
    )
    
    return model


In [47]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 10
all_scores = []
all_mae_histories = []

for i in range(k):
    print("k-fold validation #", i + 1)
    val_data = train_data[i * num_val_samples:(i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples:(i + 1) * num_val_samples]

    left_train_data = train_data[0:i * num_val_samples]
    right_train_data = train_data[(i + 1) * num_val_samples:]
    partial_train_data = np.concatenate([left_train_data, right_train_data], axis = 0)
    
    left_train_targets = train_targets[0:i * num_val_samples]
    right_train_targets = train_targets[(i + 1) * num_val_samples:]
    partial_train_targets = np.concatenate([left_train_targets, right_train_targets], axis = 0)
    
    assert len(partial_train_data) == 303
    assert len(val_data) == 101
    
    model = build_model()
    
    history = model.fit(
        partial_train_data, partial_train_targets,
        epochs = num_epochs,
        batch_size = 1,
        #verbose = 0,
        validation_data=(val_data, val_targets)
    )
    
    all_mae_histories.append(history)

k-fold validation # 1
Train on 303 samples, validate on 101 samples
Epoch 1/10
303/303 [==============================] - 62s 206ms/step - loss: 203.8534 - mean_absolute_error: 10.5296 - val_loss: 38.5267 - val_mean_absolute_error: 4.4154
Epoch 2/10
303/303 [==============================] - 63s 207ms/step - loss: 30.1066 - mean_absolute_error: 3.8496 - val_loss: 21.8943 - val_mean_absolute_error: 2.9434
Epoch 3/10
303/303 [==============================] - 62s 206ms/step - loss: 21.4138 - mean_absolute_error: 3.1931 - val_loss: 19.2176 - val_mean_absolute_error: 2.7188
Epoch 4/10
303/303 [==============================] - 63s 208ms/step - loss: 17.0210 - mean_absolute_error: 2.7692 - val_loss: 13.3416 - val_mean_absolute_error: 2.3105
Epoch 5/10
303/303 [==============================] - 63s 207ms/step - loss: 15.2958 - mean_absolute_error: 2.6209 - val_loss: 14.5145 - val_mean_absolute_error: 2.4568
Epoch 6/10
303/303 [==============================] - 63s 207ms/step - loss: 14.0861 

In [49]:
average_mae_history = [np.mean(
    [x.history["val_mean_absolute_error"][i] for x in all_mae_histories]
) for i in range(num_epochs)]
print(average_mae_history)

[4.452557821416914, 3.276942410423319, 2.985078004290267, 2.7767231015506946, 2.7091559770673808, 2.7489363433848517, 2.624492595064463, 2.734001404135534, 2.6955782291986568, 2.6265184192521738]
